In [5]:
from dotenv import load_dotenv
from openai import OpenAI
import os

In [ ]:
# .env 파일에 정의된 환경 변수 로드
load_dotenv()

True

In [ ]:
# openai api key 연결 잘 되는지 test
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "hi!"}],
    temperature=0.0,
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


#### 데이터 전처리 함수

- 리뷰 JSON 파일 로드

In [ ]:
import json

with open("./Crawling/resource/경희대글램.json", "r", encoding="utf-8") as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '회기역 근처에 있어서 접근도 너무 편리했고 호텔 내부도 깨끗했어요! 인테리어도 좋고 테이블이 있어서 너무 편했네요 또 갈게요~',
  'stars': 5,
  'date': '2025.10.05'},
 {'review': '항상 깨끗하게 잘쉬다갑니드 감사해요', 'stars': 5, 'date': '2025.10.05'},
 {'review': '항상깔끔해서좋습니다 회기근처에선 여기만가요', 'stars': 5, 'date': '2025.10.04'}]

- 좋은 평점 및 나쁜 평점 정의
    - 좋은 평점 = 별 5개
        - 366개
    - 나쁜 평점 = 별 4개 이하
        - 54개

In [ ]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r["stars"] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1
print(good_cnt, bad_cnt)

366 54


- 좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [ ]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r["stars"] == 5:
        reviews_good.append("[REVIEW_START]" + r["review"] + "[REVIEW_END]")  # 왜
    else:
        reviews_bad.append("[REVIEW_START]" + r["review"] + "[REVIEW_END]")

print(reviews_good[:3])
print(reviews_bad[:3])

['[REVIEW_START]회기역 근처에 있어서 접근도 너무 편리했고 호텔 내부도 깨끗했어요! 인테리어도 좋고 테이블이 있어서 너무 편했네요 또 갈게요~[REVIEW_END]', '[REVIEW_START]항상 깨끗하게 잘쉬다갑니드 감사해요[REVIEW_END]', '[REVIEW_START]항상깔끔해서좋습니다 회기근처에선 여기만가요[REVIEW_END]']
['[REVIEW_START]서울에서 괜찮은 숙소 찾기가 어려운데 모처럼 가성비 좋은 곳을 찾은 것 같습니다.\n공간이 협소한게 아쉽지만, 서울이라는 지리적 위치를 생각하면 나쁘지 않다는 생각이 들었어요\n여러가지 시설과 어메니티가 다 좋아서 만족합니다.[REVIEW_END]', '[REVIEW_START]전체적으로 깔끔합니다! 방도 업그레이드 해주시고 친절하셨어요. 전객실 금연인 점도 좋았습니다!\n근데 방 딱 들어가자마자 락스 냄새가...ㅠ 화장실 청소를 아주 열심히 하셨는지... 냄새가 너무 심해서 계속 환기시키구 방향제 같은 것도 뿌렸어요ㅠ 그리고 수건이 작은거 2장 큰거 한장이라 살짝 모자른감도 있었는데, \n이것말고는 모두 전체적으로 만족했습니다. 가격대비 굿[REVIEW_END]', '[REVIEW_START]객실 내부가 깔끔하고 예쁘고 스파도 넓어서 좋았습니다! 이른 입실과 늦은 퇴실로 시간이 넉넉한 점도 좋았어요\n\n다만, 스파에 뜨거운 물이 너무 약하게 나와서 아쉬웠고, 퇴실시 프론트에 직원이 없어서 좀 기다려야 했던 점이 불편했습니다.\n\n전체적으로 만족감이 더 커서 재방문의사 있습니다!\n넓은 스파 원하신다면 한번쯤 방문해보시길 추천드립니다!![REVIEW_END]']


In [ ]:
reviews_good_text = "".join(reviews_good)
reviews_bad_text = "".join(reviews_bad)
# print(reviews_good_text[:100])
# print(reviews_bad_text[:100])

[REVIEW_START]회기역 근처에 있어서 접근도 너무 편리했고 호텔 내부도 깨끗했어요! 인테리어도 좋고 테이블이 있어서 너무 편했네요 또 갈게요~[REVIEW_END][REV
[REVIEW_START]서울에서 괜찮은 숙소 찾기가 어려운데 모처럼 가성비 좋은 곳을 찾은 것 같습니다.
공간이 협소한게 아쉽지만, 서울이라는 지리적 위치를 생각하면 나쁘지 않다


- 전처리 함수 작성

In [ ]:
import datetime
from dateutil import parser


def preprocess_reviews(path="./Crawling/resource/경희대글램.json"):
    with open(path, "r", encoding="utf-8") as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6 * 30)

    for r in review_list:
        review_date_str = r["date"]
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue

        if r["stars"] == 5:
            reviews_good.append("[REVIEW_START]" + r["review"] + "[REVIEW_END]")
        else:
            reviews_bad.append("[REVIEW_START]" + r["review"] + "[REVIEW_END]")

    reviews_good_text = "\n".join(reviews_good)
    reviews_bad_text = "\n".join(reviews_bad)

    return reviews_good_text, reviews_bad_text


good, bad = preprocess_reviews()
# good[:100]

'[REVIEW_START]회기역 근처에 있어서 접근도 너무 편리했고 호텔 내부도 깨끗했어요! 인테리어도 좋고 테이블이 있어서 너무 편했네요 또 갈게요~[REVIEW_END]\n[RE'

#### 평가용 함수 작성

평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (= LLM 기반 평가)
- 비교하는 방식 vs. 점수 매기는 방식
- 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)

In [ ]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""

    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[{"role": "user", "content": eval_prompt}],
        temperature=0.0,
    )

    return completion

#### Baseline 모델 개발

In [23]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [ ]:
reviews, _ = preprocess_reviews(path="./Crawling/resource/경희대글램.json")


def summarize(reviews, prompt, temperature=0.0, model="gpt-3.5-turbo-0125"):
    prompt = prompt + "\n\n" + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )

    return completion


print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

회기역 근처에 위치한 숙소로 접근이 편리하고 내부가 깨끗하며 편안한 분위기를 제공한다. 방음이 잘되고 친절한 서비스로 자주 방문하게 된다.


In [25]:
summary_real = "청결하고 쾌적한 환경을 제공하며, 직원들의 친절한 서비스가 돋보이는 곳입니다. 방은 넓고 방음이 잘 되어 있어 편안한 휴식을 취할 수 있으며, 난방이 잘 되어 따뜻한 분위기를 유지합니다. 위치는 회기역과 가까워 접근성이 좋고, 주변에 음식점과 편의점도 있어 편리합니다. 전반적으로 만족스러운 숙박 경험을 제공하며, 재방문 의사가 높은 숙소입니다."

In [ ]:
print(
    pairwise_eval(
        reviews,
        summarize(reviews, PROMPT_BASELINE).choices[0].message.content,
        summary_real,
    )
    .choices[0]
    .message.content
)

Assistant A와 Assistant B의 답변을 비교해보면 다음과 같습니다:

1. **도움성 및 관련성**:
   - Assistant A는 숙소의 위치와 청결함, 편안함을 강조하며 다시 방문할 의향이 있다는 점을 언급했습니다.
   - Assistant B는 청결함, 쾌적함, 직원들의 친절함, 방의 넓이와 방음, 난방, 접근성, 주변 편의시설 등을 언급하며 전반적으로 만족스러운 숙박 경험을 제공한다고 했습니다.

2. **정확성**:
   - Assistant A는 리뷰의 주요 포인트를 간략하게 요약했습니다.
   - Assistant B는 리뷰에서 언급된 다양한 요소들을 더 구체적으로 다루었습니다.

3. **깊이 및 세부사항**:
   - Assistant A는 비교적 간단한 요약을 제공했습니다.
   - Assistant B는 더 많은 세부사항을 포함하여 리뷰의 다양한 측면을 다루었습니다.

4. **창의성 및 상세도**:
   - Assistant A는 기본적인 요약에 그쳤습니다.
   - Assistant B는 리뷰의 다양한 측면을 종합하여 더 풍부한 정보를 제공했습니다.

종합적으로 볼 때, Assistant B의 답변이 더 깊이 있고 상세하며, 리뷰의 다양한 측면을 잘 반영하고 있습니다. 따라서, Assistant B의 답변이 더 나은 것으로 판단됩니다.

최종 평가: [[B]]


#### 대규모 평가 스크립트

- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [ ]:
eval_count = 10

summaries_baseline = [
    summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content
    for _ in range(eval_count)
]
summaries_baseline

['- 깨끗한 숙소, 항상 만족하는 손님들\n- 회기 근처에서 최고라는 호텔로 접근성 우수\n- 친절한 사장님과 넓은 객실, 업그레이드 제공되는 좋은 서비스\n- 깔끔하고 쾌적한 숙소로 자주 이용하고 있는 손님들\n- 회기 근처에서 가성비 높은 숙소로 애용하는 손님들',
 '주변 역 접근이 편리하고 깨끗한 숙소로 만족하고 다시 방문할 예정이에요.',
 '경기역 근처에 있는 숙소는 접근성이 편리하고 내부도 깨끗하며 테이블이 있어서 편했던 것으로 요약할 수 있습니다.',
 '회기역 근처에 위치한 숙소로 접근이 편리하고 내부가 깨끗하며 테이블이 편리하다. 항상 깨끗하고 편안하게 잘 쉬다왔고, 회기 근처에서는 딱 이곳만 선택한다. 방은 좁지만 깨끗하고 요청한 조치도 빠르게 해주었으며, 담배 냄새 걱정이 없고 여성청결제도 제공되어 자주 방문한다.청소 상태가 개선될 필요가 있다.룸 업그레이드를 해주어 좋은 추억을 만들고 사장님의 친절함도 좋았으며, 티비 앞에서 편하게 먹는 것도 즐거웠다. 반드시 다시 이용할 것이며, 깔끔하고 따뜻한 숙소라 잘 쉬었다.',
 '근처 회기역에 위치한 숙소는 접근성이 좋고 깨끗하며 친절한 스탭이 있어서 편안한 투숙이 가능하며, 객실은 조금 좁지만 깔끔하고 난방도 잘 되어서 좋았다. 또한, 룸 업그레이드와 세면대, 화장실, 샤워실이 분리돼 있어 편리하게 이용할 수 있었다. 추천할 만한 숙소로서 만족스러운 경험을 하고 있다고 한다.',
 '1. 회기역 근처로 접근도 편리하고 깨끗한 숙소이며 테이블이 있는 인테리어로 편안함을 느낌.\n2. 항상 깨끗하게 잘 쉬다감 및 여성청결제 구비돼 있음.\n3. 3시간 이른 입실 요청에도 신속히 처리해주고, 깔끔한 객실 제공.\n4. 담배 냄새 걱정 없이 청결하고 여성청결제 구비돼 있어 자주 방문.\n5. 회기역 근처에서 가장 괜찮은 숙소로 친절하고 다양한 서비스 제공.',
 '1. 회기역 근처에 위치한 호텔로 접근도 편리하고 내부는 깨끗하며 편안한 분위기. \n2. 항상 깨끗하게 관리돼서 잘 쉬다가 감사한 호

In [ ]:
from tqdm import tqdm


def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if "[[A]]" in verdict_text:
            a_cnt += 1
        elif "[[B]]" in verdict_text:
            b_cnt += 1
        elif "[[C]]" in verdict_text:
            draw_cnt += 1
        else:
            print("Evaluation Error")

    return a_cnt, b_cnt, draw_cnt


wins, losses, ties = pairwise_eval_batch(
    reviews, summaries_baseline, [summary_real for _ in range(len(summaries_baseline))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:25<00:00,  2.54s/it]

Wins: 5, Losses: 5, Ties: 0


#### 모델 고도화 1 - 조건 명시

In [ ]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [
    summarize(reviews, prompt, temperature=1.0).choices[0].message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:26<00:00,  2.68s/it]

Wins: 6, Losses: 4, Ties: 0


#### 모델 고도화 2 - 입력 데이터의 품질 증가

In [ ]:
import datetime
from dateutil import parser


def preprocess_reviews(path="./Crawling/resource/경희대글램.json"):
    with open(path, "r", encoding="utf-8") as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6 * 30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r["date"]
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue
        if len(r["review"]) < 30:
            filtered_cnt += 1
            continue

        if r["stars"] == 5:
            reviews_good.append("[REVIEW_START]" + r["review"] + "[REVIEW_END]")
        else:
            reviews_bad.append("[REVIEW_START]" + r["review"] + "[REVIEW_END]")

    reviews_good = reviews_good[: min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[: min(len(reviews_bad), 50)]

    reviews_good_text = "\n".join(reviews_good)
    reviews_bad_text = "\n".join(reviews_bad)

    return reviews_good_text, reviews_bad_text


reviews, _ = preprocess_reviews()

In [ ]:
eval_count = 10
summaries = [
    summarize(reviews, prompt, temperature=1.0, model="gpt-3.5-turbo-0125")
    .choices[0]
    .message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:29<00:00,  2.96s/it]

Wins: 9, Losses: 1, Ties: 0


#### 모델 고도화 3 - Few Shot Prompting

In [ ]:
reviews_1shot, _ = preprocess_reviews("./Crawling/resource/kyunghee_glam.json")
summary_1shot = (
    summarize(reviews_1shot, prompt, temperature=0.0, model="gpt-4-turbo-2024-04-09")
    .choices[0]
    .message.content
)
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}

아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [
    summarize(reviews, prompt, temperature=1.0, model="gpt-3.5-turbo-0125")
    .choices[0]
    .message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:27<00:00,  2.73s/it]

Wins: 8, Losses: 2, Ties: 0


In [37]:
summaries

['회기역 근처에 위치한 숙소는 접근성이 좋고 내부가 깨끗합니다. 인테리어가 좋고 편안한 분위기입니다. 다시 방문할 의향이 있습니다.\n\n숙소는 깔끔하고, 1시간 이른 입실 요청에도 신속하게 응대해줍니다. 객실은 조금 좁지만 깔끔하고 만족스럽습니다.\n\n담배 냄새 걱정은 없고 여성청결제까지 구비돼 있어 청결한 환경을 제공합니다. 자주 방문하고 만족스러운 숙박을 경험합니다.\n\n룸 업그레이드와 친절한 서비스로 만족하는 고객이 많습니다. 숙소 컨디션은 항상 쾌적하고, 청결한 환경을 유지합니다.\n\n회기역 근처 호텔 중에는 관리가 잘 되어있고 청결한 느낌이 드는 곳으로 손꼽힙니다. 친절한 직원들과 편안한 침대를 경험할 수 있습니다.',
 '회기역 근처에 위치한 숙소는 접근성이 편리하고 깨끗한 내부와 좋은 인테리어를 자랑합니다. 테이블이 마련되어 편안한 분위기를 즐길 수 있어 다시 방문하고 싶은 곳입니다. 객실은 조금 좁지만 깔끔하며, 조기 입실 요청에도 신속히 응대해 만족스러운 경험을 선사해드립니다. 청결함과 친절한 사장님의 서비스로 자주 방문하는 고객들에게 실망시키지 않는 숙박을 제공하고 있습니다. 룸 업그레이드와 침대의 푹신함, 그리고 세면대와 화장실이 분리된 편의 시설은 이 숙소의 장점 중 하나로 꼽힙니다. 회기역 근처에서의 여행 혹은 숙박을 고려 중이시라면 이 곳에서 편안하고 즐거운 시간을 보내실 수 있을 것입니다.',
 '회기역 근처에 위치한 숙소는 접근성이 편리하고 내부가 매우 깨끗한 호텔이었어요. 인테리어와 서비스 모두 만족스러웠으며, 편리한 테이블이 마음에 들었습니다. 또 방문할 의향이 높은 숙소였어요.\n\n깔끔하고 좋은 숙소였습니다. 조금 일찍 입실 요청했는데 즉각적으로 도와주셨고, 객실은 좁지만 깔끔했습니다. 요샌 숙박하는데 자주 이용하는 곳입니다.',
 '이 숙소는 회기역 근처에 위치하여 접근이 편리하며 내부는 깨끗하고 인테리어가 좋습니다. 다음에 또 방문할 예정입니다. 실내는 좁지만 깔끔하고 조용하여 휴식하기에 적합한 공간입니다. 청결

In [ ]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}

아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [
    summarize(reviews, prompt_1shot, temperature=1.0, model="gpt-3.5-turbo-0125")
    .choices[0]
    .message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:27<00:00,  2.77s/it]

Wins: 10, Losses: 0, Ties: 0


In [43]:
# prompt_oneshot -> pickle로 저장
import pickle
with open("./Prompts/prompt_oneshot.pickle", "wb") as f:
    pickle.dump(prompt_1shot, f)

In [39]:
summaries

['요약:\n이 숙소는 회기역 근처에 위치해 접근성이 우수하며 내부가 깨끗하고 인테리어가 매력적이라는 호평을 받고 있습니다. 또한 룸 업그레이드와 침대의 편안함, 방음의 우수성 등을 강조하며, 청결하고 서비스가 좋아 재방문 의사를 표하는 이용객이 많습니다. 다만 일부 객실에서는 욕조의 배수 문제와 난방 문제가 지적되었으며, 프론트의 친절함과 각종 편의 시설의 충분함이 장점으로 손꼽힙니다.',
 '이 숙소는 회기역 근처에 위치하며, 접근성이 우수하고 내부가 깨끗하며 좋은 인테리어를 자랑합니다. 이용객들은 주로 룸 업그레이드와 침대의 푹신함, 방음의 우수성을 칭찬하고 있습니다. 청결도와 서비스에도 만족하며, 프론트의 친절함과 편의 시설의 충분함을 높이 평가하고 있습니다. 객실에 배수 문제와 난방 문제가 지적되었지만, 전반적으로 청결하고 쾌적한 환경을 제공하여 많은 이용객들이 재방문 의사를 나타내고 있습니다.',
 '회기역 근처에 위치한 숙소는 접근성이 편리하고 내부가 깨끗하며 인테리어가 매력적합니다. 객실은 좁지만 깔끔하여 만족스럽다는 평가를 받았습니다. 또한, 담배 냄새를 걱정할 필요가 없고 여성 청결제가 구비돼 있어 접근성이 좋은 것으로 알려져 있습니다.그러나 주차장이 조금 좁고 욕조의 배수가 느린 문제가 지적되었으며, 일부 객실의 난방 문제도 있었다고 합니다. 전반적으로 숙소는 청결하고 서비스가 좋아 재방문 의사를 표하는 이용객들이 많았습니다. 프론트 직원의 친절함과 편의 시설이 훌륭하다는 선호도가 높게 나타났습니다.',
 '요약 결과:\n회기역 근처 숙소는 접근성이 뛰어나며 내부는 깨끗하고 인테리어가 매력적하다. 일부 객실은 업그레이드되어 만족스러운 투숙 경험을 할 수 있었고, 다양한 서비스와 시설이 호평을 받았다. 그러나 일부 객실에서는 몇 가지 불편함이 지적되었으며, 고객 서비스 향상이 요구되는 부분도 있었다. 전반적으로 청결하고 친절한 분위기 속에서 즐거운 투숙이 가능했다.',
 '이 숙소는 회기역 근처에 위치하며 접근성이 우수하고 내부가 깨끗하며 인

In [ ]:
reviews_1shot, _ = preprocess_reviews("./Crawling/resource/kyunghee_glam.json")
summary_1shot = (
    summarize(reviews_1shot, prompt, temperature=0.0, model="gpt-4-turbo-2024-04-09")
    .choices[0]
    .message.content
)

In [ ]:
reviews_2shot, _ = preprocess_reviews("./Crawling/resource/kyunghee_glam.json")
summary_2shot = (
    summarize(
        reviews_2shot, prompt_1shot, temperature=0.0, model="gpt-4-turbo-2024-04-09"
    )
    .choices[0]
    .message.content
)

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}

아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [
    summarize(reviews, prompt_2shot, temperature=1.0, model="gpt-3.5-turbo-0125")
    .choices[0]
    .message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:27<00:00,  2.72s/it]

Wins: 10, Losses: 0, Ties: 0


In [45]:
with open("./Prompts/prompt_2shot.pickle", "wb") as f:
    pickle.dump(prompt_2shot, f)